In [4]:
!apt install python3.10-venv


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,474 kB of archives.
After this operation, 2,885 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.5 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.2 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.7 [5,718 B]
Fetched 2,474 kB in 0s (8,056 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 123632 files and directories currently installed.)
Pr

In [5]:
!python -m venv myenv


In [10]:
!python -m venv myenv
!source myenv/bin/activate



In [11]:
pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [13]:
!pip install torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1
!pip install -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install datasets==2.12.0 loralib==0.1.1 einops==0.6.1


  Cloning https://github.com/huggingface/transformers.git (to revision e03a9cc) to /tmp/pip-req-build-hq508b5l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hq508b5l
  Running command git checkout -q e03a9cc
  Resolved https://github.com/huggingface/transformers.git to commit e03a9cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7138427 sha256=572e09f7a7bc8f63a6b431b381c5d4a59f8892689288672da5683ffd86c3e77b
  Stored in directory: /tmp/pip-ephem-wheel-cache-acnudjpt/wheels/64/cf/8d/50ad5f8454418f3d168fea8e1384841cb7ffdb1ab2080cb3f7
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstall

In [15]:
!pip install bitsandbytes==0.39.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 39.0 MB/s eta 0:00:00


In [16]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [17]:
!gdown 1u85RQZdRTmpjGKcCc5anCMAHZ-um4DUC

Downloading...
From: https://drive.google.com/uc?id=1u85RQZdRTmpjGKcCc5anCMAHZ-um4DUC
To: /content/ecommerce-faq.json
100% 21.0k/21.0k [00:00<00:00, 48.9MB/s]


In [19]:
with open("ecommerce-faq.json") as json_file:
    data = json.load(json_file)

In [20]:
pprint(data["questions"][0], sort_dicts=False)

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top "
           'right corner of our website and follow the instructions to '
           'complete the registration process.'}


In [21]:
pprint(data["questions"][1], sort_dicts=False)

{'question': 'What payment methods do you accept?',
 'answer': 'We accept major credit cards, debit cards, and PayPal as payment '
           'methods for online orders.'}


In [22]:
pprint(data["questions"][7], sort_dicts=False)

{'question': 'What should I do if my package is lost or damaged?',
 'answer': 'If your package is lost or damaged during transit, please contact '
           'our customer support team immediately. We will initiate an '
           'investigation and take the necessary steps to resolve the issue.'}
